In [1]:
!pip install tensorforce==0.4.4

     |████████████████████████████████| 276kB 2.7MB/s 
  Found existing installation: tensorforce 0.4.0
    Uninstalling tensorforce-0.4.0:
      Successfully uninstalled tensorforce-0.4.0


In [0]:
#connect to Google Colab TPUs
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.26.91.82:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.26.91.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.26.91.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
import tensorforce
from tensorforce.environments import Environment
#from tensorforce.agents import Agent
#from tensorforce.execution import Runner
#from tensorforce.core.networks import LayeredNetwork
#import tensorforce.environments
import os
import gym
from gym import wrappers
import numpy as np
import matplotlib.pyplot as plt


In [0]:
# Modify/Set Hyperparameters for NN's 
class hasselt_hyp:
  '''
  Hyperparameters according to DDQN paper from Hasselt et al 2015
  '''
  def __init__(self):
    #self.optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00025,
                                                # momentum=0.95)
    self.learning_rate=0.00025
    self.momentum=0.95
    self.num_iterations = 20000
    self.target_update_period = 10000
    self.gamma = 0.99
    self.replay_buffer_max_length = 1000000
    self.collect_steps_per_iteration = 4 
    self.batch_size = 32
    self.log_interval = 20000
    self.num_eval_episodes = 50
    self.eval_interval = 1000000
    self.size_of_replay_memory=30000
    #calvin - I'm adding these hyperparams?
    self.epsilon = 0.1
    self.epsilon_decay_period=1000000

In [15]:
#apply wrappers
print(tensorforce.__version__)
hasselt = hasselt_hyp()
print(dir(tensorforce.core))
def create_env(game='Alien-v0'):
  print('Creating env for {}'.format(game))
  '''env = gym.make(game)
  env = wrappers.atari_preprocessing.AtariPreprocessing(env,frame_skip=1)
  env = wrappers.frame_stack.FrameStack(env,4)'''
  return env

env = create_env()

0.4.4
['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'explorations', 'memories', 'preprocessors']
Creating env for Alien-v0


NameError: ignored

In [0]:
#create network
'''The convolution network used in the experiment is exactly the one
proposed by proposed by Mnih et al. (2015), we only provide de-
tails here for completeness. The input to the network is a 84x84x4
tensor containing a rescaled, and gray-scale, version of the last four
frames. The first convolution layer convolves the input with 32 fil-
ters of size 8 (stride 4), the second layer has 64 layers of size 4
(stride 2), the final convolution layer has 64 filters of size 3 (stride
1). This is followed by a fully-connected hidden layer of 512 units.
All these layers are separated by Rectifier Linear Units (ReLu). Fi-
nally, a fully-connected linear layer projects to the output of the
network, i.e., the Q-values. The optimization employed to train the
network is RMSProp (with momentum parameter 0.95).'''

def create_network():
  layers = [dict(type='conv2d',size=32,window=8,stride=4),#(8x8x32) with stride 4
            dict(type='conv2d',size=64,window=4,stride=2),#(4x4x64) with stride 2
            dict(type='conv2d',size=64,window=3,stride=1),#(3x3x64) with stride 1
            dict(type='flatten'),
            dict(type='dense',size=512,activation='relu'),
            ]
  return layers

In [6]:
exploration=dict(
    type='decaying', unit='timesteps', decay='polynomial',
    initial_value=1.0, decay_steps=40000, final_value=0.1
)
agent = Agent.create(agent='dqn',
                     environment=env,
                     memory=hasselt.size_of_replay_memory,
                     max_episode_timesteps=3000,
                     learning_rate=hasselt.learning_rate,
                     discount=hasselt.gamma,
                     exploration=exploration,
                     target_sync_frequency=10000,
                     double_dqn=True,
                     network=dict(type='layered', layers=create_network()),
)

TypeError: ignored

In [0]:
# Train
def train_and_eval_agent(agent, env, num_episodes=100,eval_every=10):
  agent.reset()
  query = ['action-distribution-probabilities']
  max_q_probs = []
  timesteps = 0
  for episode in range(num_episodes):
      states = env.reset()
      terminal = False
      highscore=0.0
      cumulative=0.0
      while not terminal:
          actions, queried = agent.act(states=states,query=query)
          max_q_probs.append(np.max(queried))
          states, terminal, reward = env.execute(actions=actions)
          agent.observe(terminal=terminal, reward=reward)
          timesteps+=1
          cumulative+=reward
      print('Episode {}: Cumulative Reward: {} Timesteps: {}'.format(episode,cumulative,timesteps))  

      if episode%eval_every==0:
          print('Evaluating...')
          highscore = evaluate_agent(agent,env,highscore,num_episodes=5)
  return max_q_probs
# Evaluate
def evaluate_agent(agent, env, highscore, num_episodes=5):
  sum_rewards = 0.0

  for _ in range(num_episodes):
      states = env.reset()
      internals = agent.initial_internals()
      terminal = False
      while not terminal:
          actions, internals = agent.act(states=states, internals=internals, evaluation=True)
          states, terminal, reward = env.execute(actions=actions)
          sum_rewards += reward

  avg_score = sum_rewards / num_episodes
  print('Mean episode reward: {}'.format(avg_score) )
  if avg_score>highscore:
            agent.save(directory='best_agents',
                       filename='dqn-Alien-v0_best_eval.hdf5',
                       format='hdf5')
  return highscore


In [0]:
try:
  os.mkdir('best_agents')
except:
  pass

max_q_probs = train_and_eval_agent(agent,env)

Episode 0: Cumulative Reward: 280.0 Timesteps: 1245
Evaluating...
Mean episode reward: 140.0
Episode 1: Cumulative Reward: 140.0 Timesteps: 2057
Episode 2: Cumulative Reward: 140.0 Timesteps: 3054
Episode 3: Cumulative Reward: 130.0 Timesteps: 3749
Episode 4: Cumulative Reward: 230.0 Timesteps: 4851
Episode 5: Cumulative Reward: 140.0 Timesteps: 5678
Episode 6: Cumulative Reward: 300.0 Timesteps: 6770
Episode 7: Cumulative Reward: 130.0 Timesteps: 7773
Episode 8: Cumulative Reward: 220.0 Timesteps: 8674
Episode 9: Cumulative Reward: 250.0 Timesteps: 9716
Episode 10: Cumulative Reward: 120.0 Timesteps: 10326
Evaluating...
Mean episode reward: 242.0
Episode 11: Cumulative Reward: 210.0 Timesteps: 11328
Episode 12: Cumulative Reward: 150.0 Timesteps: 12354
Episode 13: Cumulative Reward: 190.0 Timesteps: 13031
Episode 14: Cumulative Reward: 320.0 Timesteps: 13918
Episode 15: Cumulative Reward: 900.0 Timesteps: 15303
Episode 16: Cumulative Reward: 260.0 Timesteps: 16072
Episode 17: Cumulati

In [0]:
runner = Runner(agent=agent,environment=env)

In [0]:
runner.run(num_episodes=10,evaluation=False)

Episodes: 100%|██████████| 10/10 [02:52, reward=320.00, ts/ep=889, sec/ep=17.81, ms/ts=20.0, agent=91.1%]

In [0]:
# Close agent and environment
agent.close()
env.close()